# Kaggle Fake News Classifier
Dataset: 

In [ ]:
# Import Libraries
import pandas as pd

In [ ]:
# Import Data
df = pd.read_csv("/content/drive/MyDrive/NLP - 3/Kaggle Fake News Clasifier/train.csv")

In [ ]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
df.shape

(20800, 5)

In [ ]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [ ]:
# Drop NA's
df = df.dropna()

In [ ]:
# Get Independent Features
X = df.drop("label", axis = 1)

# Get Dependent Feature
y = df["label"]

In [ ]:
import tensorflow as tf

In [ ]:
tf.__version__

'2.5.0'

In [ ]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [ ]:
# Vocabulary size
voc_size = 5000

### One Hot Representation

In [ ]:
messages = X.copy()

In [ ]:
messages = messages.reset_index()

In [ ]:
messages.head()

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [ ]:
import nltk
import re
from nltk.corpus import stopwords

In [ ]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# Data Preprocessing
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
  review = re.sub("[^a-zA-Z]", " ", messages["title"][i])
  review = review.lower()
  review = review.split()

  review = [stemmer.stem(word) for word in review if word not in stopwords.words("english")]
  review = " ".join(review)
  corpus.append(review)

In [ ]:
corpus[0:5]

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri']

In [ ]:
onehot_representation = [one_hot(words, voc_size) for words in corpus]

In [ ]:
onehot_representation[0:5]

[[3150, 3981, 82, 1189, 3042, 2358, 3519, 124, 2900, 2395],
 [1970, 2618, 1587, 3870, 3609, 4035, 580],
 [289, 2368, 1524, 1601],
 [3659, 504, 1919, 3757, 2698, 4301],
 [2357, 3609, 4997, 722, 334, 1404, 3609, 2303, 1426, 1307]]

### Embedding Representation

In [ ]:
sent_length = 20
embedded_docs = pad_sequences(onehot_representation, padding="pre", maxlen=sent_length)

In [ ]:
embedded_docs

array([[   0,    0,    0, ...,  124, 2900, 2395],
       [   0,    0,    0, ..., 3609, 4035,  580],
       [   0,    0,    0, ..., 2368, 1524, 1601],
       ...,
       [   0,    0,    0, ...,  955,  225, 1877],
       [   0,    0,    0, ..., 2886, 4916, 3166],
       [   0,    0,    0, ..., 3786, 1453,  114]], dtype=int32)

In [ ]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 3150,
       3981,   82, 1189, 3042, 2358, 3519,  124, 2900, 2395], dtype=int32)

In [ ]:
## Creating Model
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1, activation = "sigmoid"))
model.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________


In [ ]:
import numpy as np

In [ ]:
X_final = np.array(embedded_docs)
y_final = np.array(y)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size = 0.33, random_state = 42)

(12250, 20)

## Model Training

In [ ]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=64)

Epoch 1/10
192/192 [==============================] - 10s 10ms/step - loss: 0.3330 - accuracy: 0.8465 - val_loss: 0.1960 - val_accuracy: 0.9196
Epoch 2/10
192/192 [==============================] - 1s 8ms/step - loss: 0.1419 - accuracy: 0.9460 - val_loss: 0.1916 - val_accuracy: 0.9229
Epoch 3/10
192/192 [==============================] - 2s 8ms/step - loss: 0.1007 - accuracy: 0.9610 - val_loss: 0.2126 - val_accuracy: 0.9181
Epoch 4/10
192/192 [==============================] - 1s 8ms/step - loss: 0.0776 - accuracy: 0.9720 - val_loss: 0.2359 - val_accuracy: 0.9148
Epoch 5/10
192/192 [==============================] - 1s 8ms/step - loss: 0.0663 - accuracy: 0.9750 - val_loss: 0.2841 - val_accuracy: 0.9163
Epoch 6/10
192/192 [==============================] - 1s 8ms/step - loss: 0.0470 - accuracy: 0.9850 - val_loss: 0.3106 - val_accuracy: 0.9120
Epoch 7/10
192/192 [==============================] - 1s 8ms/step - loss: 0.0366 - accuracy: 0.9879 - val_loss: 0.3783 - val_accuracy: 0.9085
Epoc

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
y_pred = y_pred.reshape(6035)

In [ ]:
y_pred_classes = np.where(y_pred > 0.5, 1, 0)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score

In [ ]:
confusion_matrix(y_test,y_pred_classes)

array([[3078,  341],
       [ 199, 2417]])

In [ ]:
accuracy_score(y_test, y_pred_classes)

0.9105219552609777